In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = os.getenv('DATA_IGN_DIR')
pretrained_model_dir = os.getenv('PRETRAINED_MODEL_DIR')

sys.path.insert(0, '..')

In [2]:
import logging
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import random
from dataclasses import dataclass, field
from typing import Optional, List

import datasets
import numpy as np
from datasets import load_dataset, concatenate_datasets

from pprint import pprint

import evaluate
import transformers
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorWithPadding,
    EvalPrediction,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
    get_scheduler,
)

from torch.utils.data import DataLoader
import torch

from pdb import set_trace

from torch.nn import CrossEntropyLoss, MSELoss

from transformers.trainer_utils import EvalLoopOutput

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, recall_score

from tqdm import tqdm
import json
from datetime import datetime
import random
from datasets import concatenate_datasets

from transformers import EarlyStoppingCallback

import torch.nn as nn
import torch.nn.functional as F
from st_moe_distilgpt2_perplexity import STMoE_DistilGPT2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, device_count)

task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),

    'rotten_tomatoes': ("text", None),
    'imdb': ("text", None),
    'yelp_polarity': ("text", None),
    
}


eval_data_dict = {'imdb': 'test', 'yelp_polarity': 'test'}

is_glue = {"cola": True,
            "mnli": True,
            "mrpc": True,
            "qnli": True,
             "qqp": True,
             "rte": True,
            "sst2": True,
            "stsb": True,
            "wnli": True,}

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cpu 0


In [3]:
checkpoint_name = '20231224_checkpoint_2.pt'
checkpoint_path = os.path.join(pretrained_model_dir, checkpoint_name)

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('distilGPT2')
model = STMoE_DistilGPT2()
model.load_state_dict(torch.load(checkpoint_path, map_location=device))

print(model.net.transformer.config.vocab_size)

model.net.config.pad_token_id = None
model.net.resize_token_embeddings(len(tokenizer))

print(model.net.transformer.config.vocab_size)

Using pad_token, but it is not set yet.


50258
50257


In [5]:
%%time
# test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
test = load_dataset('bookcorpus')


CPU times: user 879 ms, sys: 330 ms, total: 1.21 s
Wall time: 2.93 s


In [6]:
test

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 74004228
    })
})

In [9]:
%%time
dataset = test['train'].select(range(10000))
encodings = tokenizer("\n\n".join(dataset["text"]), return_tensors="pt")

CPU times: user 721 ms, sys: 163 ms, total: 884 ms
Wall time: 883 ms


In [8]:
# dataset = load_dataset('bookcorpus')
# # subset_size = 10_000_000
# subset_size = 10_000_000
# random.seed(77)
# random_indices = random.sample(range(len(dataset['train'])), subset_size)

# test = dataset['train'].select(random_indices[:1000])

# encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

KeyboardInterrupt: 

In [ ]:
model = model.to(device)
model.eval()

max_length = model.net.config.n_positions
stride = 1024
seq_len = encodings.input_ids.size(1)

loss_fct = CrossEntropyLoss()

nlls = []
prev_end_loc = 0
for begin_loc in tqdm(range(0, seq_len, stride)):
    end_loc = min(begin_loc + max_length, seq_len)
    trg_len = end_loc - prev_end_loc  # 마지막 루프의 스트라이드 값과 다를 수 있음
    
    input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
    attention_mask = encodings.attention_mask[:, begin_loc:end_loc].to(device)

    target_ids = input_ids.clone()
    target_ids[:, :-trg_len] = -100

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        lm_logits = outputs[0]
        
        # Shift so that tokens < n predict n
        shift_logits = lm_logits[..., :-1, :].contiguous()
        shift_labels = target_ids[..., 1:].contiguous()

        set_trace()
        # Flatten the tokens
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

    nlls.append(loss)

    prev_end_loc = end_loc
    if end_loc == seq_len:
        break

ppl = torch.exp(torch.stack(nlls).mean())
print(ppl)

  0%|          | 0/162 [00:00<?, ?it/s]

> /tmp/ipykernel_1892327/4015813403.py(33)<module>()
     31         set_trace()
     32         # Flatten the tokens
---> 33         loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
     34 
     35     nlls.append(loss)



ipdb>  lm_logits.shape


torch.Size([1, 1024, 50257])


ipdb>  shift_logits.shape


...

GPT2Tokenizer(name_or_path='distilGPT2', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)


ipdb>  tokenizer.config


*** AttributeError: 'GPT2Tokenizer' object has no attribute 'config'


ipdb>  dir(tokenizer)


['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_tokens', '_additional_special_tokens', '_auto_class', '_batch_encode_plus', '_batch_prepare_for_model', '_bos_token', '_build_conversation_input_ids', '_call_one', '_cls_token', '_convert_id_to_token', '_convert_token_to_id', '_convert_token_to_id_with_added_voc', '_create_repo', '_create_trie', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eos_token', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_in_target_context_manager', 